In [4]:
import pandas as pd 
import numpy as np
import os
import glob
import mysql.connector
import datetime
from mysql.connector import errorcode
from sklearn import preprocessing
from datetime import date

import warnings
warnings.filterwarnings("ignore")

In [5]:
user = 'root'
password = ''

In [6]:
path = 'C:\\Users\\User\\Documents\\Python Scripts\\hw1' #указать свой путь хранения данных 
                                                        #(файлы excel + тетрадка ivakhnenko_hw_1.ipynb),
                                                        #заменить "\", указанный в пути по умолчанию, на "\\"

-------------------------------------------------------------------------------

## Пункт 1

In [7]:
extension = 'xlsx' #Получение списка эксель файлов
os.chdir(path)
excel_files = [i for i in glob.glob('*.{}'.format(extension))]
applications = excel_files [0:64]
contracts = excel_files[64:]

In [8]:
df = pd.read_excel(applications[0], header = None, usecols = [0, 3, 4, 5, 6, 7, 8, 12, 14]) 
#Читаем первый excel, чтобы получить названия колонок

In [9]:
df

,0,1,2,3,4,5,6,7,8
0,NaN,NaN,NaN,NaN,Loan Application Form,NaN,NaN,NaN,NaN
1,1. Pesonal Information,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06.16.2014
2,Name,NaN,NaN,NaN,Date of Birth,Gender,Employed By,NaN,Date appointed
3,CGGCCG,NaN,NaN,NaN,07.22.1968,Female,School,NaN,03.16.2011
4,Identity Number,Issue Date,Mobile Number,Home Number,Email Address,NaN,Education,NaN,NaN
5,100003,08.29.2013,NaN,NaN,NaN,NaN,Higher education,NaN,NaN
6,Street Address,NaN,Children,Family,Marital Status,NaN,City,NaN,NaN
7,NaN,NaN,0,2,Married,NaN,NaN,NaN,NaN
8,City,NaN,State,NaN,Zip Code,NaN,Business Phone,Position,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Core staff,NaN


In [10]:
columns = [df[0][4], df[0][13], df[0][15], df[0][17], df[0][19], df[1][4], df[2][6], df[2][17], df[3][6], df[4][2], df[4][6],
          df[5][2], df[6][2], df[6][4], df[7][8], 'application_date', df[8][2]]
#список с названиями колнок

In [11]:
values = []
for i in applications:
    df = pd.read_excel(i, header = None, usecols = [0, 3, 4, 5, 6, 7, 8, 12, 14])
    if df.shape[0] < 21:
        df.loc[20] = np.nan
    values.append((df[0][5], df[0][14], df[0][16], df[0][18], df[0][20], df[1][5], df[2][7], df[2][18], 
                   df[3][7], df[4][3], df[4][7], df[5][3], df[6][3], df[6][5], df[7][9], df[8][1], df[8][3]))
#значения построчно для соответсствующих колонок с добавлением NaN в поле age_of_car значений нет

In [12]:
applic_df = pd.DataFrame(data = values, columns = columns)
applic_df.head(4)

,Identity Number,Income,Income Type,Housing,Age of Car (if owned),Issue Date,Children,House ownership,Family,Date of Birth,Marital Status,Gender,Employed By,Education,Position,application_date,Date appointed
0,100003,270000.0,State servant,House / apartment,NaN,08.29.2013,0.0,N,2.0,07.22.1968,Married,Female,School,Higher education,Core staff,06.16.2014,03.16.2011
1,100004,67500.0,Working,House / apartment,26.0,01.28.2010,0.0,Y,1.0,1964-10-11 00:00:00,Single / not married,Male,Government,Secondary / secondary special,Laborers,2017-02-01 00:00:00,05.22.2016
2,100006,135000.0,Working,House / apartment,NaN,2010-08-01 00:00:00,0.0,Y,2.0,08.29.1964,Civil marriage,Female,Business Entity Type 3,Secondary / secondary special,Laborers,2016-10-09 00:00:00,05.16.2008
3,100007,121500.0,Working,House / apartment,NaN,03.18.2008,0.0,Y,1.0,1963-09-02 00:00:00,Single / not married,Male,Religion,Secondary / secondary special,Core staff,2017-05-09 00:00:00,2009-12-05 00:00:00


In [13]:
applic_df.Gender[applic_df.Gender == 'Female'] = 0
applic_df.Gender[applic_df.Gender == 'Male'] = 1

In [14]:
applic_df = applic_df.drop(['Date appointed'], axis = 1)

#### Приведение всех дат к единому формату и получение переменной age вместо date_of_birth

In [15]:
for i, value in enumerate(applic_df['Issue Date']):
    try:
        applic_df['Issue Date'].loc[i] = datetime.date.strftime(datetime.datetime.strptime(value, '%m.%d.%Y'), "%Y-%m-%d")
    except:
        applic_df['Issue Date'].loc[i] = (str(applic_df['Issue Date'][i].year) + '-' +
                                          str(applic_df['Issue Date'][i].month) + '-' + str(applic_df['Issue Date'][i].day))

In [16]:
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [17]:
ages = []
for i, value in enumerate(applic_df['Date of Birth']):
    try:
        ages.append(calculate_age(datetime.datetime.strptime(applic_df['Date of Birth'][i], '%m.%d.%Y')))
    except:
        ages.append(calculate_age(value))
        
applic_df['age'] = ages

In [18]:
for i, value in enumerate(applic_df['application_date']):
    try:
        applic_df['application_date'].loc[i] = datetime.date.strftime(datetime.datetime.strptime(value, '%m.%d.%Y'), "%Y-%m-%d")
    except:
        applic_df['application_date'].loc[i] = (str(applic_df['application_date'][i].year) + 
                                                '-' + str(applic_df['application_date'][i].month) + '-' + 
                                                str(applic_df['application_date'][i].day))

In [19]:
applic_df = applic_df.replace(np.nan, 'NULL') # меняем все наны, потому что в БД np.nan не импортируется для строк

#### Передача таблицы в БД

In [20]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS applications 
(id INT, 
income FLOAT, 
income_type VARCHAR(30), 
housing VARCHAR (30), 
age_of_car INT, 
issue_date DATE, 
children INT,
house_ownership VARCHAR(30), 
family INT,
marital_status VARCHAR(30), 
gender BOOLEAN,
employed_by VARCHAR(30), 
education VARCHAR(30),
position VARCHAR(30),
application_date DATE,
age INT,
PRIMARY KEY (id)) ENGINE=INNODB""")

In [21]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

for i in range(0, applic_df.shape[0]):
    cursor.execute(""" INSERT INTO applications
    VALUES 
    (%s, %s, '%s', '%s', %s, '%s', %s, '%s', %s, '%s', %s, '%s', '%s', '%s', '%s', '%s')""" % (
        applic_df['Identity Number'][i],
        applic_df['Income'][i],
        applic_df['Income Type'][i], 
        applic_df['Housing'][i], 
        applic_df['Age of Car (if owned)'][i], 
        applic_df['Issue Date'][i], 
        applic_df['Children'][i], 
        applic_df['House ownership'][i],
        applic_df['Family'][i], 
        applic_df['Marital Status'][i],
        applic_df['Gender'][i], 
        applic_df['Employed By'][i], 
        applic_df['Education'][i], 
        applic_df['Position'][i], 
        applic_df['application_date'][i],
        applic_df['age'][i]))

cnx.commit()
cnx.close()

#### всё тоже самое только для контрактов

In [22]:
df_contract = pd.read_excel(contracts[0], header = None, usecols = [0, 4, 5])

In [23]:
column_values_contracts = [df_contract[0][4], df_contract[0][6], df_contract[0][8], 
                           df_contract[1][4], df_contract[1][6], df_contract[1][8], 'contract_date']

In [24]:
values_contracts = []
for i in contracts:
    df = pd.read_excel(i, header = None, usecols = [0, 4, 5])
    values_contracts.append((df[0][5], df[0][7], df[0][9], df[1][5], df[1][7], df[1][9], df[2][1]))

In [25]:
contract_df = pd.DataFrame(data = values_contracts, columns = column_values_contracts)
contract_df.head(5)

,Identity Number,Amount,Term (month),Contract Number,Type,Annuity,contract_date
0,100009,1560726.0,41,100022,Cash loans,41301.0,2015-02-11 00:00:00
1,100047,1193580.0,37,100052,Cash loans,35028.0,08.30.2016
2,100025,1132573.5,33,100068,Cash loans,37561.5,12.30.2018
3,100064,298728.0,22,100113,Cash loans,15381.0,02.19.2018
4,100023,544491.0,34,100122,Cash loans,17563.5,07.31.2016


In [26]:
for i, value in enumerate(contract_df['contract_date']):
    try:
        contract_df['contract_date'].loc[i] = datetime.date.strftime(datetime.datetime.strptime(value, '%m.%d.%Y'), "%Y-%m-%d")
    except:
        contract_df['contract_date'].loc[i] = (str(contract_df['contract_date'][i].year) + 
                                                '-' + str(contract_df['contract_date'][i].month) + '-' + 
                                                str(contract_df['contract_date'][i].day))

In [27]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS contracts 
(id INT, 
amount INT, 
term_month INT, 
contract_number INT, 
type VARCHAR(30), 
annuity INT, 
contract_date DATE,
PRIMARY KEY (id),
FOREIGN KEY (id) REFERENCES applications(id)) ENGINE=INNODB """)

In [28]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

for i in range(0, contract_df.shape[0]):
    try:
        cursor.execute(""" INSERT INTO contracts
        VALUES 
        (%s, %s, %s, %s, '%s', %s, '%s')""" % (
            contract_df['Identity Number'][i],
            contract_df['Amount'][i],
            contract_df['Term (month)'][i], 
            contract_df['Contract Number'][i], 
            contract_df['Type'][i],
            contract_df['Annuity'][i],
            contract_df['contract_date'][i]))
    except:
        print("Нет заявки с id = %d, такой контракт не передаётся в базу" %contract_df['Identity Number'][i])

cnx.commit()
cnx.close()

Нет заявки с id = 100002, такой контракт не передаётся в базу


## Пункт 2

In [29]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

applications_df = pd.read_sql_query("SELECT * FROM applications", cnx)
contracts_df = pd.read_sql_query("SELECT * FROM contracts", cnx)

cnx.close()


In [30]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

cursor.execute("SELECT COUNT(*) FROM applications WHERE income < 0")
if cursor.fetchall()[0][0] > 0:
    print("Found income < 0")
    cursor.execute("UPDATE applications SET income=-1*(income) WHERE income < 0")
    print("income < 0 changed into positive")
else:
    print("No negative values for income")
  
cursor.execute("SELECT MIN(income) FROM applications")
print('min value income =', cursor.fetchall()[0][0])    
cursor.execute("SELECT MAX(income) FROM applications")
print('max value income =', cursor.fetchall()[0][0])
print('-' * 50)


    
cursor.execute("SELECT COUNT(*) FROM applications WHERE family < 0")
if cursor.fetchall()[0][0] > 0:
    print("Found family < 0")
    cursor.execute("UPDATE applications SET family=-1*(family) WHERE family < 0")
    print("family < 0 changed into positive")
else:
    print("No negative values for family")
    
cursor.execute("SELECT MIN(family) FROM applications")
print('min value family =', cursor.fetchall()[0][0])    
cursor.execute("SELECT MAX(family) FROM applications")
print('max value family =', cursor.fetchall()[0][0])
print('-' * 50)



cursor.execute("SELECT COUNT(*) FROM applications WHERE children < 0")
if cursor.fetchall()[0][0] > 0:
    print("Found children < 0")
    cursor.execute("UPDATE applications SET children=-1*(children) WHERE children < 0")
    print("children < 0 changed into positive")
else:
    print("No negative values for children")
    
cursor.execute("SELECT MIN(children) FROM applications")
print('min value children =', cursor.fetchall()[0][0])    
cursor.execute("SELECT MAX(children) FROM applications")
print('max value children =', cursor.fetchall()[0][0])
print('-' * 50)
    
cursor.execute("SELECT COUNT(*) FROM applications WHERE age < 0")
if cursor.fetchall()[0][0] > 0:
    print("Found age < 0")
    cursor.execute("UPDATE applications SET age=-1*(age) WHERE age < 0")
    print("age < 0 changed into positive")
else:
    print("No negative values for age")
    
cursor.execute("SELECT MIN(age) FROM applications")
print('min value age =', cursor.fetchall()[0][0])    
cursor.execute("SELECT MAX(age) FROM applications")
print('max value age =', cursor.fetchall()[0][0])
print('-' * 50)

cursor.execute("SELECT MIN(issue_date) FROM applications")
print('min value issue_date =', cursor.fetchall()[0][0])    
cursor.execute("SELECT MAX(issue_date) FROM applications")
print('max value issue_date =', cursor.fetchall()[0][0])
print('-' * 50)


cursor.execute("SELECT MIN(application_date) FROM applications")
print('min value application_date =', cursor.fetchall()[0][0])    
cursor.execute("SELECT MAX(application_date) FROM applications")
print('max value application_date =', cursor.fetchall()[0][0])
print('-' * 50)  




No negative values for income
min value income = 38419.15625
max value income = 540000.0
--------------------------------------------------
No negative values for family
min value family = 1
max value family = 4
--------------------------------------------------
No negative values for children
min value children = 0
max value children = 2
--------------------------------------------------
No negative values for age
min value age = 21
max value age = 71
--------------------------------------------------
min value issue_date = 2001-10-01
max value issue_date = 2017-03-28
--------------------------------------------------
min value application_date = 2014-05-06
max value application_date = 2018-12-29
--------------------------------------------------


In [31]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

print("Checking types of income_type")
cursor.execute("SELECT COUNT(income_type), income_type FROM applications GROUP BY income_type")
for i in cursor.fetchall():
    print(i)   
print("_" * 100)

print("Checking types of housing")
cursor.execute("SELECT COUNT(housing), housing FROM applications GROUP BY housing")
for i in cursor.fetchall():
    print(i)   
print("_" * 100)

print("Checking types of marital_status")
cursor.execute("SELECT COUNT(marital_status), marital_status FROM applications GROUP BY marital_status")
for i in cursor.fetchall():
    print(i)   
print("_" * 100)

print("Checking types of employed_by")
cursor.execute("SELECT COUNT(employed_by), employed_by FROM applications GROUP BY employed_by")
for i in cursor.fetchall():
    print(i)   
print("_" * 100)

print("Checking types of education")
cursor.execute("SELECT COUNT(education), education FROM applications GROUP BY education")
for i in cursor.fetchall():
    print(i)   
print("_" * 100)



cnx.commit()
  

    
    
print("Checking types of position")
cursor.execute("SELECT COUNT(position), position FROM applications GROUP BY position")
for i in cursor.fetchall():
    print(i)
print('-'*20)
print("Found '<undefined>' for position")
cursor.execute("""UPDATE applications SET position = 'NULL' WHERE position = '<undefined>'""")
print("'<undefined>' changed into NULL")
print("_" * 100)



Checking types of income_type
(8, 'State servant')
(37, 'Working')
(9, 'Commercial associate')
(9, 'Pensioner')
(1, 'NULL')
____________________________________________________________________________________________________
Checking types of housing
(58, 'House / apartment')
(2, 'Rented apartment')
(2, 'With parents')
(1, 'Municipal apartment')
(1, 'NULL')
____________________________________________________________________________________________________
Checking types of marital_status
(39, 'Married')
(12, 'Single / not married')
(8, 'Civil marriage')
(3, 'Widow')
(1, 'Separated')
(1, 'NULL')
____________________________________________________________________________________________________
Checking types of employed_by
(2, 'School')
(3, 'Government')
(12, 'Business Entity Type 3')
(1, 'Religion')
(3, 'Other')
(9, 'XNA')
(1, 'Electricity')
(2, 'Medicine')
(3, 'Business Entity Type 2')
(8, 'Self-employed')
(2, 'Transport: type 2')
(1, 'Construction')
(1, 'Housing')
(1, 'Kindergarten

In [32]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

cursor.execute("SELECT COUNT(*) FROM contracts WHERE amount < 0")
if cursor.fetchall()[0][0] > 0:
    print("Found amount < 0")
    cursor.execute("UPDATE contracts SET amount=-1*(amount) WHERE amount < 0")
    print("amount < 0 changed into positive")
else:
    print("No negative values for amount")
  
cursor.execute("SELECT MIN(amount) FROM contracts")
print('min value amount =', cursor.fetchall()[0][0])    
cursor.execute("SELECT MAX(amount) FROM contracts")
print('max value amount =', cursor.fetchall()[0][0])
print('-' * 50)


    
cursor.execute("SELECT COUNT(*) FROM contracts WHERE term_month < 0")
if cursor.fetchall()[0][0] > 0:
    print("Found term_month < 0")
    cursor.execute("UPDATE contracts SET term_month=-1*(term_month) WHERE term_month < 0")
    print("term_month < 0 changed into positive")
else:
    print("No negative values for term_month")
  
cursor.execute("SELECT MIN(term_month) FROM contracts")
print('min value term_month =', cursor.fetchall()[0][0])    
cursor.execute("SELECT MAX(term_month) FROM contracts")
print('max value term_month =', cursor.fetchall()[0][0])
print('-' * 50)

cursor.execute("SELECT COUNT(*) FROM contracts WHERE annuity < 0")
if cursor.fetchall()[0][0] > 0:
    print("Found annuity < 0")
    cursor.execute("UPDATE contracts SET annuity=-1*(annuity) WHERE annuity < 0")
    print("annuity < 0 changed into positive")
else:
    print("No negative values for annuity")
  
cursor.execute("SELECT MIN(annuity) FROM contracts")
print('min value annuity =', cursor.fetchall()[0][0])    
cursor.execute("SELECT MAX(annuity) FROM contracts")
print('max value annuity =', cursor.fetchall()[0][0])
print('-' * 50)

No negative values for amount
min value amount = 80865
max value amount = 1663988
--------------------------------------------------
No negative values for term_month
min value term_month = 12
max value term_month = 41
--------------------------------------------------
No negative values for annuity
min value annuity = 5882
max value annuity = 86990
--------------------------------------------------


In [33]:
cursor.execute("SELECT COUNT(*) FROM contracts WHERE amount > annuity * term_month")
if cursor.fetchall()[0][0] > 0:
    print("Error: amount > annuity * term_month")
else:
    print("No error: amount < annuity * term_month")

No error: amount < annuity * term_month


## Пункт 3

In [34]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

applications_df = pd.read_sql_query("SELECT * FROM applications", cnx) # Тянем таблицы из базы обратно
contracts_df = pd.read_sql_query("SELECT * FROM contracts", cnx)

cnx.close()

In [35]:
le = preprocessing.LabelEncoder() #Используем встроеннуб функцию кодировки
le.fit(applications_df['income_type'])
numbers = le.transform(applications_df['income_type'])

In [36]:
ids = applications_df.id
id_numbers = list(zip(ids, numbers)) #Лист из id и значений чтобы обновлять таблицу по id

In [37]:
keys = le.inverse_transform(numbers)
keys_and_numbers = dict(zip(keys, numbers)) #Словарь ключей и значений

In [38]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()
    
cursor.execute("""ALTER TABLE applications ADD income_type_enc INT""") #Создаём колонку для перекодированной переменной
    
for i in id_numbers:
    cursor.execute("""UPDATE applications SET income_type_enc = {0} WHERE id = {1} """.format(i[1], i[0])) #обновляем колонку
    
cursor.execute("CREATE TABLE keys_applications (feature VARCHAR(30), keys_f VARCHAR(30), values_f INT)") #Таблица ключей и значений

for i in keys_and_numbers:
    cursor.execute(""" INSERT INTO keys_applications
            VALUES 
            ('%s', '%s', %d)""" % (
            'income_type',
            i,
            keys_and_numbers[i]))
    
cnx.commit()

#### Те же операции для всех колонок с  текстовыми полями

In [39]:
le = preprocessing.LabelEncoder()
le.fit(applications_df['housing'])
numbers = le.transform(applications_df['housing'])

ids = applications_df.id
id_numbers = list(zip(ids, numbers))
keys = le.inverse_transform(numbers)
keys_and_numbers = dict(zip(keys, numbers))

cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()
    
cursor.execute("""ALTER TABLE applications ADD housing_enc INT""")
    
for i in id_numbers:
    cursor.execute("""UPDATE applications SET housing_enc = {0} WHERE id = {1} """.format(i[1], i[0]))
    
for i in keys_and_numbers:
    cursor.execute(""" INSERT INTO keys_applications
            VALUES 
            ('%s', '%s', %d)""" % (
            'housing',
            i,
            keys_and_numbers[i]))
    
cnx.commit()



In [40]:
le = preprocessing.LabelEncoder()
le.fit(applications_df['marital_status'])
numbers = le.transform(applications_df['marital_status'])

ids = applications_df.id
id_numbers = list(zip(ids, numbers))
keys = le.inverse_transform(numbers)
keys_and_numbers = dict(zip(keys, numbers))

cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()
    
cursor.execute("""ALTER TABLE applications ADD marital_status_enc INT""")
    
for i in id_numbers:
    cursor.execute("""UPDATE applications SET marital_status_enc = {0} WHERE id = {1} """.format(i[1], i[0]))
    
for i in keys_and_numbers:
    cursor.execute(""" INSERT INTO keys_applications
            VALUES 
            ('%s', '%s', %d)""" % (
            'marital_status',
            i,
            keys_and_numbers[i]))
    
cnx.commit()



In [41]:
le = preprocessing.LabelEncoder()
le.fit(applications_df['employed_by'])
numbers = le.transform(applications_df['employed_by'])

ids = applications_df.id
id_numbers = list(zip(ids, numbers))
keys = le.inverse_transform(numbers)
keys_and_numbers = dict(zip(keys, numbers))

cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()
    
cursor.execute("""ALTER TABLE applications ADD employed_by_enc INT""")
    
for i in id_numbers:
    cursor.execute("""UPDATE applications SET employed_by_enc = {0} WHERE id = {1} """.format(i[1], i[0]))
    
for i in keys_and_numbers:
    cursor.execute(""" INSERT INTO keys_applications
            VALUES 
            ('%s', '%s', %d)""" % (
            'employed_by',
            i,
            keys_and_numbers[i]))
    
cnx.commit()



In [42]:
le = preprocessing.LabelEncoder()
le.fit(applications_df['position'])
numbers = le.transform(applications_df['position'])

ids = applications_df.id
id_numbers = list(zip(ids, numbers))
keys = le.inverse_transform(numbers)
keys_and_numbers = dict(zip(keys, numbers))

cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()
    
cursor.execute("""ALTER TABLE applications ADD position_enc INT""")
    
for i in id_numbers:
    cursor.execute("""UPDATE applications SET position_enc = {0} WHERE id = {1} """.format(i[1], i[0]))
    
for i in keys_and_numbers:
    cursor.execute(""" INSERT INTO keys_applications
            VALUES 
            ('%s', '%s', %d)""" % (
            'position',
            i,
            keys_and_numbers[i]))
    
cnx.commit()


In [43]:
le = preprocessing.LabelEncoder()
le.fit(applications_df['house_ownership'])
numbers = le.transform(applications_df['house_ownership'])

ids = applications_df.id
id_numbers = list(zip(ids, numbers))
keys = le.inverse_transform(numbers)
keys_and_numbers = dict(zip(keys, numbers))

cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()
    
cursor.execute("""ALTER TABLE applications ADD house_ownership_enc INT""")
    
for i in id_numbers:
    cursor.execute("""UPDATE applications SET house_ownership_enc = {0} WHERE id = {1} """.format(i[1], i[0]))
    
for i in keys_and_numbers:
    cursor.execute(""" INSERT INTO keys_applications
            VALUES 
            ('%s', '%s', %d)""" % (
            'house_ownership',
            i,
            keys_and_numbers[i]))
    
cnx.commit()



In [44]:
le = preprocessing.LabelEncoder()
le.fit(applications_df['education'])
numbers = le.transform(applications_df['education'])

ids = applications_df.id
id_numbers = list(zip(ids, numbers))
keys = le.inverse_transform(numbers)
keys_and_numbers = dict(zip(keys, numbers))

cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()
    
cursor.execute("""ALTER TABLE applications ADD education_enc INT""")
    
for i in id_numbers:
    cursor.execute("""UPDATE applications SET education_enc = {0} WHERE id = {1} """.format(i[1], i[0]))
    
for i in keys_and_numbers:
    cursor.execute(""" INSERT INTO keys_applications
            VALUES 
            ('%s', '%s', %d)""" % (
            'education',
            i,
            keys_and_numbers[i]))
    
cnx.commit()



In [45]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

cursor.execute("ALTER TABLE applications DROP income_type")
cursor.execute("ALTER TABLE applications DROP housing")
cursor.execute("ALTER TABLE applications DROP house_ownership")
cursor.execute("ALTER TABLE applications DROP marital_status")
cursor.execute("ALTER TABLE applications DROP employed_by")
cursor.execute("ALTER TABLE applications DROP education")
cursor.execute("ALTER TABLE applications DROP position")


cnx.commit()
cnx.close()

#### В контрактах только одна колонка с текстовыми значениями

In [46]:
le = preprocessing.LabelEncoder()
le.fit(contracts_df['type'])
numbers = le.transform(contracts_df['type'])

ids = contracts_df.id
id_numbers = list(zip(ids, numbers))
keys = le.inverse_transform(numbers)
keys_and_numbers = dict(zip(keys, numbers))

cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()
    
cursor.execute("""ALTER TABLE contracts ADD type_enc INT""")
    
for i in id_numbers:
    cursor.execute("""UPDATE contracts SET type_enc = {0} WHERE id = {1} """.format(i[1], i[0]))
    
cursor.execute("CREATE TABLE keys_contracts (feature VARCHAR(30), keys_f VARCHAR(30), values_f INT)")
    
for i in keys_and_numbers:
    cursor.execute(""" INSERT INTO keys_contracts
            VALUES 
            ('%s', '%s', %d)""" % (
            'type',
            i,
            keys_and_numbers[i]))
    
cnx.commit()



In [47]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw_db', user = user, password = password)
cursor = cnx.cursor()

cursor.execute("ALTER TABLE contracts DROP type")



cnx.commit()
cnx.close()